<a href="https://colab.research.google.com/github/yuya1201/Digital-Advertising-HW/blob/master/Python_Exercise_4_Using_CensusReporter_to_Make_Geographic_Selections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import requests
import json
import urllib
from urllib.request import Request,urlopen
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# load data
df=pd.read_csv('drive/My Drive/Colab Notebooks/msas.csv')
df.head()

,CBSA,NAME,POPESTIMATE2017
0,10180,"Abilene, TX",170219
1,10420,"Akron, OH",703505
2,10500,"Albany, GA",151434
3,10540,"Albany, OR",125047
4,10580,"Albany-Schenectady-Troy, NY",886188


In [0]:
# dived data into half and half
geoid='31000US'+df.CBSA.astype('str')
h1=geoid[:len(geoid)//2]
h2=geoid[len(geoid)//2:]

h1=','.join(h1)
h2=','.join(h2)

# 1. What city has the highest count of the audience you chose for parameter #1?

### Knowing that we're going after Organic consumers, what average HH income should we choose?  

We could pick columns $50k to 74.9k 'B19001011' and 'b19001012'.


According to the calculation below, New York-Newark-Jersey City, NY-NJ-PA has the highest count of average house hold income that ranged between 50k to 74.9k, which is 1.01121e+06.


In [0]:
# first parameter
tableid1=['B19001']
tableidstring1=','.join(tableid1)
# iterate throuth first half data and get right data
requesturl11 = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=%s' % (tableidstring1, h1)
loadedjson11 = requests.get(requesturl11)
parsedjson11 = json.loads(loadedjson11.text)
# iterate throuth second half data and get right data
requesturl12 = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=%s' % (tableidstring1, h2)
loadedjson12 = requests.get(requesturl12)
parsedjson12 = json.loads(loadedjson12.text)

In [0]:
# create a new dataframe
df1=pd.DataFrame(columns=['country','population'],index=list(range(1,len(geoid))))

In [0]:
# put first half data into the new dataframe
for i in list(range(1,len(geoid)//2)):
  df1['country'][i]=geoid[i]
  df1['population'][i]=parsedjson11['data'][geoid[i]]['B19001']['estimate']['B19001011']+parsedjson11['data'][geoid[i]]['B19001']['estimate']['B19001012']

In [0]:
# put second half data into the new dataframe
for i in list(range(len(geoid)//2,len(geoid))):
  df1['country'][i]=geoid[i]
  df1['population'][i]=parsedjson12['data'][geoid[i]]['B19001']['estimate']['B19001011']+parsedjson12['data'][geoid[i]]['B19001']['estimate']['B19001012']

In [0]:
# find the country with CBSA code
print(df1.loc[df1.population==max(df1.population)])

          country   population
249  31000US35620  1.01121e+06


In [0]:
# find the location using CBSA code
print(df.loc[df.CBSA==35620])

      CBSA                                   NAME  POPESTIMATE2017
249  35620  New York-Newark-Jersey City, NY-NJ-PA         20320876


# 2. What city has the highest count of the audience you chose for parameter #2?

### What age range should we target?

We could choose age range 18-34:  
'B01001007','B01001008','B01001009','B01001010','B01001011','B01001012'
  

Based on the calculation down below, New York-Newark-Jersey City, NY-NJ-PA has the highest count of age range 18-34, which is 2.33054e+06.

In [0]:
# second parameter
tableid2=['B01001']
tableidstring2=','.join(tableid2)
# iterate throuth first half data and get right data
requesturl21 = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=%s' % (tableidstring2, h1)
loadedjson21 = requests.get(requesturl21)
parsedjson21 = json.loads(loadedjson21.text)
parsedjson11 = json.loads(loadedjson11.text)
# iterate throuth second half data and get right data
requesturl22 = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=%s' % (tableidstring2, h2)
loadedjson22 = requests.get(requesturl22)
parsedjson22 = json.loads(loadedjson22.text)

In [0]:
# create a list of choosed censuses
agerange=['B01001007','B01001008','B01001009','B01001010','B01001011','B01001012']
# create a new dataframe
df2=pd.DataFrame(columns=['location','population'],index=list(range(1,len(geoid))))

In [0]:
# put first half data into the new dataframe
for i in list(range(1,len(geoid)//2)):
  df2['location'][i]=geoid[i]
  df2['population'][i]=parsedjson21['data'][geoid[i]]['B01001']['estimate'][agerange[0]]+ \
    parsedjson21['data'][geoid[i]]['B01001']['estimate'][agerange[1]]+ \
    parsedjson21['data'][geoid[i]]['B01001']['estimate'][agerange[2]]+ \
    parsedjson21['data'][geoid[i]]['B01001']['estimate'][agerange[3]]+ \
    parsedjson21['data'][geoid[i]]['B01001']['estimate'][agerange[4]]+ \
    parsedjson21['data'][geoid[i]]['B01001']['estimate'][agerange[5]]
# put second half data into the new dataframe
for i in list(range(len(geoid)//2,len(geoid))):
  df2['location'][i]=geoid[i]
  df2['population'][i]=parsedjson22['data'][geoid[i]]['B01001']['estimate'][agerange[0]]+ \
    parsedjson22['data'][geoid[i]]['B01001']['estimate'][agerange[1]]+ \
    parsedjson22['data'][geoid[i]]['B01001']['estimate'][agerange[2]]+ \
    parsedjson22['data'][geoid[i]]['B01001']['estimate'][agerange[3]]+ \
    parsedjson22['data'][geoid[i]]['B01001']['estimate'][agerange[4]]+ \
    parsedjson22['data'][geoid[i]]['B01001']['estimate'][agerange[5]]

In [0]:
# find the country with CBSA code
print(df2.loc[df2.population==max(df2.population)])

         location   population
249  31000US35620  2.33054e+06


In [0]:
# find the location using CBSA code
print(df.loc[df.CBSA==35620])

      CBSA                                   NAME  POPESTIMATE2017
249  35620  New York-Newark-Jersey City, NY-NJ-PA         20320876


# 3. What city has the highest mean percentage across all three categories?

### Should we look for areas with large Hispanic populations?

The hispanic code would be B03003003.

According to the calculation down below,  Los Angeles-Long Beach-Anaheim, CA has the highest count of hispanic, which is 6.00354e+06.


In [0]:
# third parameter
tableid3=['B03003']
tableidstring3=','.join(tableid3)
# iterate throuth first half data and get right data
requesturl31 = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=%s' % (tableidstring3, h1)
loadedjson31 = requests.get(requesturl31)
parsedjson31 = json.loads(loadedjson31.text)
# iterate throuth second half data and get right data
requesturl32 = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=%s' % (tableidstring3, h2)
loadedjson32 = requests.get(requesturl32)
parsedjson32 = json.loads(loadedjson32.text)

In [0]:
df3=pd.DataFrame(columns=['location','population'],index=list(range(1,len(geoid))))

In [0]:
# put first half data into the new dataframe
for i in list(range(1,len(geoid)//2)):
  df3['location'][i]=geoid[i]
  df3['population'][i]=parsedjson31['data'][geoid[i]]['B03003']['estimate']['B03003003']
# put second half data into the new dataframe
for i in list(range(len(geoid)//2,len(geoid))):
  df3['location'][i]=geoid[i]
  df3['population'][i]=parsedjson32['data'][geoid[i]]['B03003']['estimate']['B03003003']

In [0]:
# find the country with CBSA code
print(df3.loc[df3.population==max(df3.population)])

         location   population
210  31000US31080  6.00354e+06


In [0]:
# find the location using CBSA code
print(df.loc[df.CBSA==31080])

      CBSA                                NAME  POPESTIMATE2017
210  31080  Los Angeles-Long Beach-Anaheim, CA         13353907


# 4. What city has the highest mean percentage across all three categories?

New York-Newark-Jersey City, NY-NJ-PA has the highest percentage across all three categories, which is 0.940082.

In [0]:
# create list for eaach question above
q1_percent = []
q2_percent = []
q3_percent = []
# calculate percentages and put each of them into the corresponding list.
for i, row in df1.iterrows():
  q1_percent.append(df1.population[i] / max(df1.population))
  q2_percent.append(df2.population[i] / max(df2.population))
  q3_percent.append(df3.population[i] / max(df3.population))

In [0]:
# combine the list with corresponding dataframe.
df1['percentage'] = q1_percent
df2['percentage'] = q2_percent
df3['percentage'] = q3_percent

In [0]:
# calculate percentage mean
percentmean = []
for i, row in df1.iterrows():
  mean = (df1.percentage[i] + df2.percentage[i] + df3.percentage[i]) / 3
  percentmean.append(mean)
df1['percentage mean'] = percentmean

In [0]:
# find the record with largest percentage mean
print(df1.loc[df1['percentage mean'] == max(df1['percentage mean'])])

          country   population  percentage  percenage mean  percentage mean
249  31000US35620  1.01121e+06         1.0        0.940082         0.940082


In [0]:
# find the location with the largest percentage mean
print(df.loc[df.CBSA == 35620])

      CBSA                                   NAME  POPESTIMATE2017
249  35620  New York-Newark-Jersey City, NY-NJ-PA         20320876
